# WhisPeriment
Experimenting with the Whisper model
- Record an audio sequence
- Allow to replay it
- Provide the text in Engilsh

In [1]:
# conda create --name hugg python=3.10 -y
# conda activate hugg 
# conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia -y
# conda install transformers diffusers["torch"] pyaudio -y
# conda install numpy scipy ipywidgets ipykernel -y
# conda install accelerate -y


In [2]:
import math
import struct
import pyaudio
import wave

from transformers    import pipeline, __version__ 
from IPython.display import Audio, display

print(f"pyaudio      : {pyaudio.__version__}")
print(f"wave         : Nov version infos available")
print(f"transformers : {__version__ }")

pyaudio      : 0.2.12
wave         : Nov version infos available
transformers : 4.24.0


# Load the Whisper model
Choose the model depending of execution time and Available GPU mem:
- 12G -> large 'Note there has been a recent improvment of the large model)
-  6G -> medium

In [3]:
whisper = pipeline('automatic-speech-recognition', model = 'openai/whisper-medium', device =0)

In [10]:
short_normalize   = (1.0/32768.0)
chunk             = 1024
swidth            = 2 
frames_per_buffer = 12800
format            = pyaudio.paInt16
channels          = 1
sample_rate       = 44100
audio_file        = 'record.wav'

def rms(frame):
  """Return the RMS value of the frame content"""
  count       = len(frame) / swidth
  format      = "%dh" % (count)
  shorts      = struct.unpack(format, frame)
  sum_squares = 0.0
  for sample in shorts:
      n            = sample * short_normalize
      sum_squares += n * n
  rms = math.pow(sum_squares / count, 0.5)

  return rms * 1000

430.6640625


In [11]:
t_record_s          = 10
n_chucnk            = int(t_record_s * sample_rate / chunk)
p                   = pyaudio.PyAudio()
stream              = p.open(    
   format           = format,
   channels         = channels,
   rate             = sample_rate,
   input            = True,
   frames_per_buffer= frames_per_buffer
)
frames = []
print(f"-----Now Recording for {t_record_s} s-----")
for i in range(0,400):
  audio_data   = stream.read(chunk, exception_on_overflow = False)
  rms_val      = rms(audio_data)
  frames.append(audio_data)

print(f'-----End Recording----- Last RMS: {rms_val:6.2f} ')
stream.stop_stream()    # Stop Audio Recording  IMPORTANT
stream.close()          # Close Audio Recording IMPORTANT

wf = wave.open(audio_file, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(format))
wf.setframerate(sample_rate)
wf.writeframes(b''.join(frames))
wf.close()

-----Now Recording for 10 s-----
-----End Recording----- Last RMS:   1.55 


In [6]:

display(Audio(audio_file, autoplay=True))

In [12]:
text = whisper(audio_file)
print(text)

/home/me/miniforge3/envs/hugg/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': " During the next 10 seconds, this program will record what I say. And what I say too. Yes, it's true too."}
